In [4]:
import os.path
import re
import json
import pandas as pd
import datetime
import numpy as np

import geopandas as gpd
import folium
from folium import IFrame

import plotly
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import dash_table
from dash.exceptions import PreventUpdate
from jupyter_dash import JupyterDash

pd.set_option('display.max_columns', None) 
pd.options.mode.chained_assignment = None

In [5]:
df_full = pd.read_csv(os.path.join('..', 'Data', 'xrysi_eykairia', 'Full', 'full_dataset.csv'))
municipalities = gpd.read_file(os.path.join('..', 'Data', 'geojson', 'output.geojson'))

In [6]:
df_full['construction_era'] = pd.cut(df_full['construction_year'], 
                                     bins=[0, 1970, 1980, 1990, 2000, 2010, 2020, 2030], 
                                     labels=['< 1970', '1970-1980', '1980-1990', '1990-2000', '2000-2010', '2010-2020', '> 2020'])

df_full['area_group'] = pd.cut(df_full['area'], 
                                     bins=[0, 20, 40, 60, 10000], 
                                     labels=['< 20', '21-40', '41-60', '61+'])

df_full['bedrooms_group'] = pd.cut(df_full['bedrooms'], 
                                   bins=[0, 1, 2, 3, 10000], 
                                   labels=['1', '2', '3', '4+'])

df_full['level_group'] = pd.cut(df_full['min_level'], 
                                bins=[-100, -1, -0.5, 0, 0.5, 1, 2, 10000], 
                                labels=['Υπόγειο', 'Ημιυπόγειο', 'Ισόγειο', 'Ημιώροφος', '1', '2', '3+'])


In [7]:
daily_ids = {date: set(df_full.loc[df_full['extraction_date'] == date, 'id']) for date in df_full['extraction_date'].unique()}
df_population = pd.DataFrame(daily_ids.items(), columns=['extraction_date','ids'])
df_population['ids_yesterday'] = df_population['ids'].shift(1)
df_population.dropna(inplace=True)
df_population['in'] = (df_population['ids'] - df_population['ids_yesterday']).apply(len)
df_population['out'] = (df_population['ids_yesterday'] - df_population['ids']).apply(len)

In [8]:
fig_bar_population_new_left = px.bar(df_population, x='extraction_date', y=['in', 'out'], orientation='v', 
                                     barmode="group")
fig_bar_population_new_left.update_traces(marker_color='darkcyan', selector=dict(type='bar', name='in'))
fig_bar_population_new_left.update_traces(marker_color='darkturquoise', selector=dict(type='bar', name='out'))

fig_bar_population_new_left.update_traces(text=df_population['in'], textposition='auto', selector=dict(type='bar', name='in'))
fig_bar_population_new_left.update_traces(text=df_population['out'], textposition='auto', selector=dict(type='bar', name='out'))

fig_bar_population_new_left.update_layout(
    title="Incoming and outgoing ads",
    title_font=dict(size=24),
    title_x=0.5
)

fig_bar_population_new_left.update_layout(
    xaxis_title='Date',
    yaxis_title='Ad Count'
)

df_ad_age = df_full[df_full['extraction_date'] == "2023-06-05"].groupby('days_passed').size().reset_index(name='Count')
df_ad_age['age'] = pd.cut(df_ad_age['days_passed'], bins=[0, 7, 14, 21, 28, 60], labels=['< 1 week', '< 2 weeks', '< 3 weeks', '< 1 month', '< 2 months'])
df_age_graph = df_ad_age.groupby('age')['Count'].sum().reset_index(name='Count')
fig_ad_age = px.bar(df_age_graph, x='age', y='Count', 
             orientation='v', title='Ad age (2023-06-05)')
fig_ad_age.update_traces(marker_color='darkcyan')
fig_ad_age.update_traces(text=df_age_graph['Count'], textposition='auto')
fig_ad_age.update_layout(
    xaxis_title='Age',
    yaxis_title='Count',
    title_x=0.5
)

None

In [12]:
# app = JupyterDash(__name__, external_stylesheets=['styles_test.css'])
# app = JupyterDash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app = JupyterDash(__name__)
cols_to_keep = ['extraction_date', 'dimos', 'perioxi', 'type',  'price', 'area', 'bedrooms', 'bathrooms', 'min_level', 'construction_year']

page1_layout = html.Div(
    className="container",
    children=[
        html.H1("Data exploration"),
        html.Div(
            style={
                "display": "inline-block",
                "verticalAlign": "top",
                "width": "20%",
                "padding": "10px",
            },
            children=[
                html.Label("Ημερομηνία:", style={'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='date-filter-1',
                    options=[
                        {'label': str(date), 'value': date}
                        for date in df_full['extraction_date'].unique()
                    ],
                    value=None
                ),
                html.Br(),
                
                html.Label("Δήμος:", style={'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='dimos-filter-1',
                    options=[
                        {'label': dimos, 'value': dimos}
                        for dimos in df_full['dimos'].unique()
                    ],
                    value=None
                ),
                html.Br(),
                
#                 html.Label("Περιοχή:", style={'font-weight': 'bold'}),
#                 dcc.Dropdown(
#                     id='perioxi-filter-1',
#                     options=[
#                         {'label': perioxi, 'value': perioxi}
#                         for perioxi in df_full['perioxi'].unique()
#                     ],
#                     value=None
#                 ),
#                 html.Br(),
                
                html.Label("Είδος Κατοικίας:", style={'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='type-filter-1',
                    options=[
                        {'label': dimos, 'value': dimos}
                        for dimos in df_full['type'].unique()
                    ],
                    value=None
                ),
                html.Br(),
                
                html.Label("Τιμή:", style={'font-weight': 'bold'}),
                dcc.RangeSlider(
                    id='price-range-slider-1',
                    min=100,
#                     max=df_full['price'].max(),
                    max=2000,
                    step=50,
                    value=[300, 600],
#                     marks={100: '100', int(df_full['price'].max()): str(int(df_full['price'].max()))},
                    marks={100: '100', 2000: '2000'},
                ),
                html.Div(id='selected-range-price-1'),
                html.Br(),
                
                html.Label("Εμβαδόν:", style={'font-weight': 'bold'}),
                dcc.RangeSlider(
                    id='area-range-slider-1',
                    min=0,
#                     max=df_full['area'].max(),
                    max=500,
                    step=20,
                    value=[40, 80],
#                     marks={int(value): str(int(value)) for value in [0, df_full['area'].max()]},
                    marks={int(value): str(int(value)) for value in [0, 500]},
                ),
                html.Div(id='selected-range-area-1'),
                html.Br(),
                
                html.Label("Υπνοδωμάτια:", style={'font-weight': 'bold'}),
                dcc.RangeSlider(
                    id='bedrooms-range-slider-1',
                    min=0,
                    max=df_full['bedrooms'].max(),
                    step=1,
                    value=[1, 2],
                    marks={int(value): str(int(value)) for value in [0, df_full['bedrooms'].max()]},
                ),
                html.Div(id='selected-range-bedrooms-1'),
                html.Br(),
                
                html.Label("Μπάνια:", style={'font-weight': 'bold'}),
                dcc.RangeSlider(
                    id='bathrooms-range-slider-1',
                    min=0,
                    max=df_full['bathrooms'].max(),
                    step=1,
                    value=[1, 2],
                    marks={int(value): str(int(value)) for value in [0, df_full['bathrooms'].max()]},
                ),
                html.Div(id='selected-range-bathrooms-1'),
                html.Br(),
                
                html.Label("Όροφος:", style={'font-weight': 'bold'}),
                dcc.RangeSlider(
                    id='levels-range-slider-1',
                    min=df_full['min_level'].min(),
                    max=df_full['min_level'].max(),
                    step=1,
                    value=[df_full['min_level'].min(), df_full['min_level'].max()],
                    marks={int(value): str(int(value)) for value in [0, df_full['min_level'].max()]},
                ),
                html.Div(id='selected-range-levels-1'),
                html.Br(),
                
                html.Label("Έτος Κατασκευής:", style={'font-weight': 'bold'}),
                dcc.RangeSlider(
                    id='construction_year-range-slider-1',
                    min=df_full['construction_year'].min(),
                    max=df_full['construction_year'].max(),
                    step=1,
                    value=[1970, df_full['construction_year'].max()],
                    marks={int(value): str(int(value)) for value in [df_full['construction_year'].min(), df_full['min_level'].max()]},
                ),
                html.Div(id='selected-range-construction_year-1'),
                html.Br(),
            ],
        ),
        html.Div(
            style={
                "display": "inline-block",
                "width": "70%",
                "padding": "10px",
            },
            children=[
                dash_table.DataTable(
                    id='table',
#                     columns=[{'name': col, 'id': col} for col in df.columns],
                    columns=[{'name': greek_col, 'id': col} for greek_col, col in zip(['Ημερομηνία', 'Δήμος', 'Περιοχή', 'Είδος Κατοικίας', "Τιμή", 'Εμβαδόν', 'Υπνοδωμάτια', 'Μπάνια', 'Όροφος', 'Έτος Κατασκευής',  'Link'], cols_to_keep)],
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'minWidth': '0px', 'maxWidth': '180px',
                        'whiteSpace': 'normal',
                        'textAlign': 'left',
                        'userSelect': 'text',
                    },
                ),
                html.Br(),
                html.Iframe(id='map-sites-1', srcDoc = open(os.path.join('..', 'Data', 'other', 'default_maps', 'temp_ads.html'), 'r').read(), width='100%', height='450'),
                html.Br(),
#                 dcc.Graph(id='fig-population', figure=fig_bar_population_new_left),
#                 html.Br(),
#                 dcc.Graph(id='fig-ad-age', figure=fig_ad_age),
#                 html.Br(),
                dcc.Link('Go to Page 2', href='/page2'),
            ],
        ),
    ],
)

page2_layout = html.Div(
    className="container",
    children=[
        html.H1("Price driving factors"),
        html.Div(
            style={
                "display": "inline-block",
                "verticalAlign": "top",
                "width": "20%",
                "padding": "10px",
            },
            children=[
                html.Label("Μορφή χάρτη:", style={'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='map-type-filter-2',
                    options=[
                        {'label': map_type, 'value': map_type}
                        for map_type in ['Τιμή', 'Προσφορά']
                    ],
                    value='Τιμή'
                ),
                html.Br(),
                
                html.Label("Ημερομηνία:", style={'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='date-filter-2',
                    options=[
                        {'label': str(date), 'value': date}
                        for date in df_full['extraction_date'].unique()
                    ],
                    value='2023-06-05'
                ),
                html.Br(),
                
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                
                html.Label("Παράμετρος:", style={'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='graphtype-filter-2',
                    options=[
                        {'label': x, 'value': x}
                        for x in ['Όροφος', 'Πλήθος ορόφων', 'Πλήθος υπνοδωματίων', 'Έτος Κατασκευής', 'Εμβαδόν']
                    ],
                    value='Όροφος'
                ),
                html.Br(),
                
                html.Label("Δήμος:", style={'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='dimos-filter-2',
                    options=[
                        {'label': dimos, 'value': dimos}
                        for dimos in df_full['dimos'].unique()
                    ],
                    value=None
                ),
                html.Br(),
                
                html.Label("Είδος Κατοικίας:", style={'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='type-filter-2',
                    options=[
                        {'label': x, 'value': x}
                        for x in df_full['type'].unique()
                    ],
                    value=None
                ),
                html.Br(),
                
                html.Label("Εμβαδόν:", style={'font-weight': 'bold'}),
                dcc.Dropdown(
                    id='areagroup-filter-2',
                    options=[
                        {'label': x, 'value': x}
                        for x in df_full['area_group'].unique()
                    ],
                    value=None
                ),
                html.Br(),
            ],
        ),
        html.Div(
            style={
                "display": "inline-block",
                "width": "70%",
                "padding": "10px",
            },
            children=[
                html.Div(id='map-title-label-2'),
                html.Iframe(id='map-price', srcDoc = open(os.path.join('..', 'Data', 'other', 'default_maps', 'temp_choropleth.html'), 'r').read(), width='100%', height='450'),
                html.Br(),
                dcc.Graph(id='fig-2'),
                html.Br(),
                dcc.Link('Go to Page 1', href='/page1'),
            ],
        ),
    ],
)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

@app.callback(
    Output('selected-range-price-1', 'children'),
    Input('price-range-slider-1', 'value')
)
def update_selected_range_price(value):
    return f'Επιλεγμένο εύρος τιμής: {value[0]}€ - {value[1]}€'

@app.callback(
    Output('selected-range-area-1', 'children'),
    Input('area-range-slider-1', 'value')
)
def update_selected_range_area(value):
    return f'Επιλεγμένο εύρος εμβαδόν: {value[0]}m² - {value[1]}m²'

@app.callback(
    Output('selected-range-bedrooms-1', 'children'),
    Input('bedrooms-range-slider-1', 'value')
)
def update_selected_range_bedrooms(value):
    return f'Επιλεγμένο εύρος υπνοδωματίων: {value[0]} - {value[1]}'

@app.callback(
    Output('selected-range-bathrooms-1', 'children'),
    Input('bathrooms-range-slider-1', 'value')
)
def update_selected_range_area_bathrooms(value):
    return f'Επιλεγμένο εύρος μπάνιων: {value[0]} - {value[1]}'

@app.callback(
    Output('selected-range-levels-1', 'children'),
    Input('levels-range-slider-1', 'value')
)
def update_selected_range_levels(value):
    return f'Επιλεγμένο εύρος ορόφων: {value[0]} - {value[1]}'

@app.callback(
    Output('selected-range-construction_year-1', 'children'),
    Input('construction_year-range-slider-1', 'value')
)
def update_selected_range_construction_year(value):
    return f'Επιλεγμένο εύρος έτους κατασκευής: {value[0]} - {value[1]}'

@app.callback(
    [Output('table', 'data'),
    Output('map-sites-1', 'srcDoc')],
    [Input('date-filter-1', 'value'), 
     Input('dimos-filter-1', 'value'),
#      Input('perioxi-filter-1', 'value'),
     Input('type-filter-1', 'value'), 
     Input('price-range-slider-1', 'value'), 
     Input('area-range-slider-1', 'value'), 
     Input('bedrooms-range-slider-1', 'value'), 
     Input('bathrooms-range-slider-1', 'value'), 
     Input('levels-range-slider-1', 'value'), 
     Input('construction_year-range-slider-1', 'value')
    ]
)
def update_df_1(date_filter_value, dimos_filter_value, type_filter_value, price_filter_tuple, area_filter_tuple, bedrooms_filter_tuple, bathrooms_filter_tuple, level_filter_tuple, construction_year_filter_tuple):
    
    filtered_df = df_full[cols_to_keep + ['geo_lat', 'geo_lng', 'url']]
    filtered_df = filtered_df[(price_filter_tuple[0] <= filtered_df['price']) & (filtered_df['price'] <= price_filter_tuple[1])]
    filtered_df = filtered_df[(area_filter_tuple[0] <= filtered_df['area']) & (filtered_df['area'] <= area_filter_tuple[1])]
    filtered_df = filtered_df[(bedrooms_filter_tuple[0] <= filtered_df['bedrooms']) & (filtered_df['bedrooms'] <= bedrooms_filter_tuple[1])]
    filtered_df = filtered_df[(bathrooms_filter_tuple[0] <= filtered_df['bathrooms']) & (filtered_df['bathrooms'] <= bathrooms_filter_tuple[1])]
    filtered_df = filtered_df[(level_filter_tuple[0] <= filtered_df['min_level']) & (filtered_df['min_level'] <= level_filter_tuple[1])]
    filtered_df = filtered_df[(construction_year_filter_tuple[0] <= filtered_df['construction_year']) & (filtered_df['construction_year'] <= construction_year_filter_tuple[1])]
    
    if date_filter_value:
        filtered_df = filtered_df[filtered_df['extraction_date'] == date_filter_value]
        
    if dimos_filter_value:
        filtered_df = filtered_df[filtered_df['dimos'] == dimos_filter_value]
        
#     if perioxi_filter_value:
#         filtered_df = filtered_df[filtered_df['perioxi'] == perioxi_filter_value]
        
    if type_filter_value:
        filtered_df = filtered_df[filtered_df['type'] == type_filter_value]
        
#     def generate_link(url):
#         return html.A('link', href=url, target='_blank')
#     filtered_df['link'] = filtered_df['url'].apply(lambda x: generate_link(x))

    final_df = filtered_df.head(20)
    tooltip_columns = ['price', 'area', 'bedrooms', 'construction_year']

    map_athens = folium.Map(location=[final_df['geo_lat'].iloc[0], final_df['geo_lng'].iloc[0]], zoom_start=16)
    for _, row in final_df.iterrows():
        x = '<br>'.join([f"{column.capitalize()}: {int(row[column])}" for column in tooltip_columns])
#         x = x.replace(row['url'], f'<a href="{row["url"]}" target="_blank">Link</a>')
#         x = x.replace(row['url'], f'<a href="{row["url"]}" target="_blank" onclick="event.stopPropagation();">{row["url"]}</a>')
        folium.Marker(
            location=[row['geo_lat'], row['geo_lng']],
#             tooltip=folium.Popup(x, max_width=300),
            tooltip=x,
            sticky=True,
            icon=folium.Icon(color='blue', icon='home')
        ).add_to(map_athens)
    
    return final_df[cols_to_keep].to_dict('records'), map_athens.get_root().render()

@app.callback(
    [Output('map-price', 'srcDoc'), Output('map-title-label-2', 'value')],
    [Input('date-filter-2', 'value'), Input('map-type-filter-2', 'value')]
)
def update_page_2_map(date_filter_value, map_type_value):
    df_filtered = df_full[df_full['extraction_date'] == date_filter_value]
    dimos_df = df_filtered.groupby('dimos').agg({'price_per_area': 'mean', 'kwd_ypes': 'first', 'id': 'count'}).reset_index()
    dimos_df.columns = ['dimos', 'price_per_area', 'kwd_ypes', 'count']
    dimos_df['kwd_ypes'] = dimos_df['kwd_ypes'].astype(str)
    map_df = municipalities.merge(dimos_df, left_on="kwd_ypes", right_on="kwd_ypes", how="outer")
    
    if map_type_value == "Τιμή":
        important_cols = ['kwd_ypes', 'price_per_area']
        label_title = 'Τιμή ανά τετραγωνικό μέτρο'
    else:
        important_cols = ['kwd_ypes', 'count']
        label_title = 'Πλήθος αγγελιών'
    
    map_athens = folium.Map(location=[37.9838, 23.7275], zoom_start=11, tiles='openstreetmap')
    folium.Choropleth(
            geo_data=os.path.join('..', 'Data', 'geojson', 'output.geojson'),
            data=map_df,
            columns=important_cols,  #Here we tell folium to get the county fips and plot new_cases_7days metric for each county
            key_on='feature.properties.kwd_ypes', #Here we grab the geometries/county boundaries from the geojson file using the key 'coty_code' which is the same as county fips
            #threshold_scale=custom_scale, #use the custom scale we created for legend
            fill_color='YlOrRd',
            nan_fill_color="White", #Use white color if there is no data available for the county
            fill_opacity=0.8,
            line_opacity=0.2,
            legend_name='Price per unit area', #title of the legend
            highlight=True,
            line_color='black').add_to(map_athens)
    
    return map_athens.get_root().render(), label_title


@app.callback(
    Output('fig-2', 'figure'),
    [Input('graphtype-filter-2', 'value'), 
    Input('date-filter-2', 'value'), 
    Input('dimos-filter-2', 'value'),
    Input('type-filter-2', 'value'),
    Input('areagroup-filter-2', 'value')]
)
def update_page_2_graph(graph_type_value, date_filter_value, dimos_filter_value, type_filter_value, area_group_value):
    if date_filter_value is None:
        raise PreventUpdate
    
    df_filtered = df_full[(df_full['extraction_date'] == date_filter_value) & (df_full['level_length'] > 0)]
    if dimos_filter_value:
        df_filtered = df_filtered[df_filtered['dimos'] == dimos_filter_value]
        
    if type_filter_value:
        df_filtered = df_filtered[df_filtered['type'] == type_filter_value]
        
    if area_group_value:
        df_filtered = df_filtered[df_filtered['area_group'] == area_group_value]
        
    if graph_type_value == 'Όροφος':
    
        df1 = df_filtered.groupby('level_group').aggregate({'price_per_area': ['mean', 'count']}).reset_index()
        df1.columns = ['level', 'price_per_area', 'count']

        fig = px.bar(df1, x='price_per_area', y='level', orientation='h', hover_data='count')

        fig.update_traces(marker_color='darkcyan')
        fig.update_traces(text=df1['price_per_area'], textposition='auto', texttemplate='%{text:.4s}')
        fig.update_layout(
            xaxis_title='Average price per area',
            yaxis_title='Level',
            title='Prices for different house levels',
            title_x=0.5
        )
        
    if graph_type_value == 'Πλήθος ορόφων':
    
        df2 = df_filtered.groupby('level_length').aggregate({'price_per_area': ['mean', 'count']}).reset_index()
        df2.columns = ['level_length', 'price_per_area', 'count']
        fig = px.bar(df2, x='price_per_area', y='level_length', orientation='h', hover_data='count')

        fig.update_traces(marker_color='darkcyan')
        fig.update_traces(text=df2['price_per_area'], textposition='auto', texttemplate='%{text:.4s}')
        fig.update_layout(
            xaxis_title='Average price per area',
            yaxis_title='Levels',
            title='Do house levels affect price per area?',
            title_x=0.5
        )
    
    if graph_type_value == 'Πλήθος υπνοδωματίων':
        
        df_bedrooms = df_filtered.groupby('bedrooms_group').agg({'price_per_area': 'mean', 'id': 'count'}).reset_index()
        df_bedrooms.columns = ['bedrooms', 'price_per_area', 'count']
        
        fig = px.bar(df_bedrooms, x='bedrooms', y='price_per_area', orientation='v', hover_data='count')

        fig.update_traces(marker_color='darkcyan')
        fig.update_traces(text=df_bedrooms['price_per_area'], textposition='auto', texttemplate='%{text:.4s}')
        fig.update_layout(
            xaxis_title='Number of Bedrooms',
            yaxis_title='Average price per area',
            title='How do bedrooms affect price?',
            title_x=0.5
        )

    if graph_type_value == 'Έτος Κατασκευής':
        
        df_price_by_year = df_filtered.groupby('construction_era').agg({'price_per_area': 'mean', 'id': 'count'}).reset_index()
        df_price_by_year.columns = ['construction_era', 'price_per_area', 'count']
        
        fig = px.bar(df_price_by_year, x='construction_era', y='price_per_area', orientation='v', hover_data='count')

        fig.update_traces(marker_color='darkcyan')
        fig.update_traces(text=df_price_by_year['price_per_area'], textposition='auto', texttemplate='%{text:.4s}')
        fig.update_layout(
            xaxis_title='Construction Period',
            yaxis_title='Average price per area',
            title='Does construction period affect price?',
            title_x=0.5
        )

    if graph_type_value == 'Εμβαδόν':
        
        df_area_group = df_filtered.groupby('area_group').agg({'price_per_area': 'mean', 'id': 'count'}).reset_index()
        df_area_group.columns = ['area_group', 'price_per_area', 'count']
        
#         fig_area_group = px.bar(df_area_group, x='area_group', y='count', orientation='v', hover_data='price_per_area')
        fig = px.bar(df_area_group, x='area_group', y='price_per_area', orientation='v', hover_data='count')

        fig.update_traces(marker_color='darkcyan')
        fig.update_traces(text=df_area_group['count'], textposition='auto')
        fig.update_layout(
            xaxis_title='Area Group',
            yaxis_title='Average price per area',
            title='Does area affect price per area?',
            title_x=0.5
        )
        
    if graph_type_value in ['Όροφος', 'Πλήθος ορόφων']:
        fig.update_xaxes(range=[0, 25])

    return fig

@app.callback(
    dash.dependencies.Output('page-content', 'children'),
    [dash.dependencies.Input('url', 'pathname')]
)
def display_page(pathname):
    href_dict = {'/': page1_layout,
                '/page1': page1_layout,
                '/page2': page2_layout}
    return href_dict[pathname]

app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



In [22]:
df_full.groupby(['dimos'])['perioxi'].nunique()

dimos
Άγιοι Ανάργυροι - Καματερό      3
Άγιος Δημήτριος                 1
Ίλιον                           9
Αθηναίων                      101
Αιγάλεω                         3
Βύρωνας                         7
Γαλάτσι                         7
Γλυφάδα                         1
Δάφνη - Υμηττός                 8
Ζωγράφου                        5
Ηλιούπολη                       1
Καισαριανή                      6
Καλλιθέα                       11
Μαρούσι                         1
Νέα Ιωνία                       4
Νέα Σμύρνη                      5
Παπάγου - Χολαργός              2
Περιστέρι                       9
Φιλαδέλφεια - Χαλκηδόνα         2
Φιλοθέης - Ψυχικού              4
Χαλάνδρι                        3
Name: perioxi, dtype: int64

In [62]:
df_full.columns

Index(['id', 'internal_id', 'title_abbreviation', 'address', 'price',
       'price_per_unit_area', 'size_with_square_meter', 'construction_year',
       'levels', 'bedrooms', 'bathrooms', 'date', 'extraction_date', 'geo_lat',
       'geo_lng', 'company_title', 'account_id', 'ad_group_id', 'url', 'dimos',
       'perioxi', 'type', 'area', 'price_per_area', 'level_list', 'min_level',
       'level_length', 'max_level', 'a', 'temp_no', 'temp_date_unit',
       'days_passed', 'kwd_ypes', 'enotites'],
      dtype='object')

In [63]:
df_full['geo_lat']

0        37.979777
1        38.017481
2        37.992291
3        37.999083
4        37.990937
           ...    
27108    37.987296
27109    37.949763
27110    37.949668
27111    37.950313
27112    37.986158
Name: geo_lat, Length: 27113, dtype: float64

In [7]:
map_athens = folium.Map(location=[df_full['geo_lat'].iloc[0], df_full['geo_lng'].iloc[0]], zoom_start=12)
for _, row in df_full.head(10).iterrows():
    folium.Marker(
        location=[row['geo_lat'], row['geo_lng']],
        tooltip=row['price'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_athens)
map_athens.save('map0.html')